In [55]:
import pandas as pd
from gurobipy import *
import openpyxl

In [56]:
# reading the inputs from Excel workbook
wb = openpyxl.load_workbook("Model_Inputs.xlsx")
main = wb['Main']

In [57]:
# read objective
obj = main.cell(column=2, row=1).value
if obj =='max':
    obj = -1
elif obj =='min':
    obj = 1
else:
    raise NameError('Objective cannot be recognized')
# read obj coefficient
obj_coeff=[]
for c in range(2,10000):
    v = main.cell(column=c, row=3).value
    if v is None:
        break
    else:
        obj_coeff.append(v)
n_variables = len(obj_coeff)

#read variable properties
variable_name = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=2).value
    if v is None:
        variable_name.append('x'+str(c+1))
    elif type(v) != str:
        variable_name.append(str(v))
    else:
        variable_name.append(v)        
        
variable_type = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=4).value
    if not v:
        t = 'C' 
    elif v == 'cont':
        t = 'C'
    elif v == 'int':
        t = 'I'
    elif v == 'bin':
        t = 'B'
    else:
        raise NameError('Variable type cannot be recognized')
    variable_type.append(t)
    
variable_lb = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=5).value
    if v is None:
        t = 0
    elif v == '"+inf"':
        raise NameError('Variable lower bound cannot be +inf')
    elif v == '"-inf"':
        t = -GRB.INFINITY
    elif type(v) in [float, int]:
        t = v
    else:
        raise NameError('Variable lower bound cannot be recognized')
    variable_lb.append(t)

variable_ub = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=6).value
    if v is None:
        t = GRB.INFINITY
    elif v == '"+inf"':
        t = GRB.INFINITY
    elif v == '"-inf"':
        raise NameError('Variable upper bound cannot be -inf')
    elif type(v) in [float, int]:
        t = v
    else:
        raise NameError('Variable upper bound cannot be recognized')
    variable_ub.append(t)
    
# whether to run sensitivity analysis
run_sa = main.cell(column=2, row=7).value

In [58]:
# read constraints
## find constraint column index
for c in range(1, n_variables+100):
    v = main.cell(column=c, row=8).value
    if v == 'constraint type':
        const_typ_cind = c
        break

if const_typ_cind-2 != n_variables:
    raise NameError('Number of variables does not match the constraints')
    
## find last constraint row index
for r in range(9, 100000):
    v = main.cell(column=const_typ_cind, row=r).value
    if v is None:
        break
    elif v not in ["<=", ">=", "="]:
        print(v)
        raise NameError("Wrong constraint type")
    else:
        last_r_consraints=r

## read constraints as dataframe
for r in range(9, last_r_consraints+1):
    v = main.cell(column=const_typ_cind+2, row=r).value
    if v is None:
        main.cell(column=const_typ_cind+2, row=r).value = "c"+str(r-8)
    elif type(v) != str:
        main.cell(column=const_typ_cind+2, row=r).value = str(v)
    else:
        1==1

values = []
cons_df = pd.DataFrame()
for r in range(9, last_r_consraints+1):
    r_values = []
    for c in range(2, const_typ_cind+3):
        r_values.append(main.cell(column=c, row=r).value)
    values.append(r_values)
    cons_df = pd.DataFrame(values)
    cons_df.fillna(0, inplace=True)
    cons_df.rename(columns={const_typ_cind-2:'Constraint Type', 
                            const_typ_cind-1:'RHS values',
                            const_typ_cind:'Constraint Name'}, inplace=True)

In [59]:
# setup model
m = Model()

In [60]:
# add variables
x=m.addVars(n_variables)
# set types, lb, ub of variables
for i in range(n_variables):
    x[i].setAttr('VarNAME', variable_name[i])
    x[i].setAttr('vType', variable_type[i])
    x[i].setAttr('lb', variable_lb[i])
    x[i].setAttr('ub', variable_ub[i])

In [61]:
# set objective
objective = quicksum(obj_coeff[i] * x[i] for i in range(n_variables))
m.setObjective(objective, obj)

In [62]:
# add constraints
if cons_df.shape[0]:
    ## add le constraints
    le_const = cons_df[cons_df['Constraint Type'] == '<=']
    le_const.reset_index(inplace=True, drop=True)
    for i in range(le_const.shape[0]):
        m.addConstr(quicksum(le_const.iloc[i,j]* x[j] for j in range(n_variables)) <= le_const['RHS values'][i], name=le_const['Constraint Name'][i])
    ## add ge constraints
    ge_const = cons_df[cons_df['Constraint Type'] == '>=']
    ge_const.reset_index(inplace=True, drop=True)
    for i in range(ge_const.shape[0]):
        m.addConstr(quicksum(ge_const.iloc[i,j]* x[j] for j in range(n_variables)) >= ge_const['RHS values'][i], name=ge_const['Constraint Name'][i])
    ## add eq constraints
    eq_const = cons_df[cons_df['Constraint Type'] == '=']
    eq_const.reset_index(inplace=True, drop=True)
    for i in range(eq_const.shape[0]):
        m.addConstr(quicksum(eq_const.iloc[i,j]* x[j] for j in range(n_variables)) == eq_const['RHS values'][i], name=eq_const['Constraint Name'][i])

In [63]:
# sensitivity analysis
def sensitivity_analysis():
    for v in m.getVars():
        print("For Variable " + v.VarName+ ":")
        print("Minimum value coefficient can take before the optimal decision changes "  + "is " + str(v.SAObjLow))
        print("Maximum value coefficient can take before the optimal decision changes "  + "is " + str(v.SAObjUp))
    
    for c in m.getConstrs():
        print("For constraint " + c.ConstrName+ ":")
        print("Shadow Price is " + str(c.pi))
        print("Minimum value RHS can take before the shadow price changes "  + "is " + str(c.SARHSLow))
        print("Maximum value RHS can take before the shadow price changes "  + "is " + str(c.SARHSUp))

def main(m, run_sa):
    m.optimize()
    if run_sa==1:
        if "I" in variable_type or "B" in variable_type:
            raise NameError('There is at least one variable with type of integer/binary.')
        else:
            sensitivity_analysis()
    else:
        1==1

In [64]:
## run the model
main(m, run_sa)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xd981af52
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [5e+00, 7e+00]
  Bounds range     [1e+02, 4e+02]
  RHS range        [1e+03, 2e+03]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8000000e+31   1.750000e+30   2.800000e+01      0s
       2    4.0400000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.040000000e+03
For Variable tables:
Minimum value coefficient can take before the optimal decision changes is 3.75
Maximum value coefficient can take before the optimal decision changes is 10.0
For Variable chairs:
Minimum value coefficient can take before the optimal decision ch

In [65]:
# print optimal objective value
m.objVal

4040.0

In [66]:
# print optimal decisions (if not listed, that means their optimal value = 0)
m.printAttr('X')


    Variable            X 
-------------------------
      tables          320 
      chairs          360 
